In [72]:
#импорт библиотек
import numpy as np #для матричных вычислений
import pandas as pd #для анализа и предобработки данных
import matplotlib.pyplot as plt #для визуализации
import seaborn as sns #для визуализации
from functools import partial

from sklearn import linear_model #линейные моделиё
from sklearn import tree #деревья решений
from sklearn import ensemble #ансамбли
from sklearn import metrics #метрики
from sklearn import preprocessing #предобработка
from sklearn.model_selection import train_test_split #сплитование выборки
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
import hyperopt
from hyperopt import hp, fmin, tpe, Trials
print("Версия Hyperopt : {}".format(hyperopt.__version__))
import optuna
from sklearn.model_selection import StratifiedKFold

%matplotlib inline
plt.style.use('seaborn')
from sklearn.linear_model import Ridge

Версия Hyperopt : 0.2.7


C:\Users\Acer\AppData\Local\Temp\ipykernel_7260\2441569993.py:24: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn')


In [73]:
data = pd.read_csv('data/_train_sem09.csv')
data.head()

,Activity,D1,D2,D3,D4,D5,D6,D7,D8,D9,...,D1767,D1768,D1769,D1770,D1771,D1772,D1773,D1774,D1775,D1776
0,1,0.000000,0.497009,0.10,0.0,0.132956,0.678031,0.273166,0.585445,0.743663,...,0,0,0,0,0,0,0,0,0,0
1,1,0.366667,0.606291,0.05,0.0,0.111209,0.803455,0.106105,0.411754,0.836582,...,1,1,1,1,0,1,0,0,1,0
2,1,0.033300,0.480124,0.00,0.0,0.209791,0.610350,0.356453,0.517720,0.679051,...,0,0,0,0,0,0,0,0,0,0
3,1,0.000000,0.538825,0.00,0.5,0.196344,0.724230,0.235606,0.288764,0.805110,...,0,0,0,0,0,0,0,0,0,0
4,0,0.100000,0.517794,0.00,0.0,0.494734,0.781422,0.154361,0.303809,0.812646,...,0,0,0,0,0,0,0,0,0,0


In [74]:
X = data.drop(['Activity'], axis=1)
y = data['Activity']

In [75]:
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state = 1, test_size = 0.2)

In [76]:
#Создаем объект класса логистическая регрессия
log_reg_easy = linear_model.LogisticRegression(max_iter = 50)
#Обучаем модель, минимизируя logloss
log_reg_easy.fit(X_train, y_train)
print("accuracy на тестовом наборе: {:.2f}".format(log_reg_easy.score(X_test, y_test)))
y_test_pred = log_reg_easy.predict(X_test)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))

accuracy на тестовом наборе: 0.76
f1_score на тестовом наборе: 0.79


c:\Users\Acer\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [77]:
#Создаем объект класса случайный лес
RandomForest_easy = ensemble.RandomForestClassifier(random_state=42)

#Обучаем модель
RandomForest_easy.fit(X_train, y_train)
#Выводим значения метрики 
y_train_pred = RandomForest_easy.predict(X_train)
print('Train: {:.2f}'.format(metrics.f1_score(y_train, y_train_pred)))
y_test_pred = RandomForest_easy.predict(X_test)
print('Test: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))

Train: 1.00
Test: 0.81


### <center> **GridSearchCV**

In [79]:
param_grid = {'penalty': ['l2', 'none'] ,#тип регурялизации
              'solver': ['lbfgs', 'saga'], #алгоритм оптимизации
              }
grid_search = GridSearchCV(
    estimator=linear_model.LogisticRegression(
        random_state=42, #генератор случайных чисел
        max_iter=20 #количество итераций на сходимость
    ), 
    param_grid=param_grid, 
    cv=5, 
    n_jobs = -1
)  
%time grid_search.fit(X_train, y_train) 
print("accuracy на тестовом наборе: {:.2f}".format(grid_search.score(X_test, y_test)))
y_test_pred = grid_search.predict(X_test)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))
print("Наилучшие значения гиперпараметров: {}".format(grid_search.best_params_))

c:\Users\Acer\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_logistic.py:1173: FutureWarning: `penalty='none'`has been deprecated in 1.2 and will be removed in 1.4. To keep the past behaviour, set `penalty=None`.
  warnings.warn(


CPU times: total: 2.42 s
Wall time: 56.2 s
accuracy на тестовом наборе: 0.76
f1_score на тестовом наборе: 0.78
Наилучшие значения гиперпараметров: {'penalty': 'none', 'solver': 'saga'}


c:\Users\Acer\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


**Попробуем расширить сетку гиперпараметров.**

In [80]:
param_grid = [
              {'penalty': ['l2', 'none'] , # тип регуляризации
              'solver': ['lbfgs', 'sag'], # алгоритм оптимизации
               'C': [0.01, 0.1, 0.3, 0.5, 0.7, 0.9, 1]}, # уровень силы регурялизации
              
              {'penalty': ['l1', 'l2'] ,
              'solver': ['liblinear', 'saga'],
               'C': [0.01, 0.1, 0.3, 0.5, 0.7, 0.9, 1]}
]
grid_search_1 = GridSearchCV(
    estimator=linear_model.LogisticRegression(random_state=1, max_iter=20), 
    param_grid=param_grid, 
    cv=5, 
    n_jobs = -1
)  
%time grid_search_1.fit(X_train, y_train) 
print("accuracy на тестовом наборе: {:.2f}".format(grid_search_1.score(X_test, y_test)))
y_test_pred = grid_search_1.predict(X_test)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))
print("Наилучшие значения гиперпараметров: {}".format(grid_search_1.best_params_))

CPU times: total: 6.02 s
Wall time: 7min 1s
accuracy на тестовом наборе: 0.77
f1_score на тестовом наборе: 0.79
Наилучшие значения гиперпараметров: {'C': 0.3, 'penalty': 'l1', 'solver': 'saga'}


c:\Users\Acer\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


**Метрику удалось улучшить, а время потратили много, в 7 раз больше!**

In [81]:
param_grid = {'n_estimators': list(range(80, 200, 30)),
              'min_samples_leaf': [5],
              'max_depth': list(np.linspace(20, 40, 5, dtype=int))
              }
            
grid_search_forest = GridSearchCV(
    estimator=ensemble.RandomForestClassifier(random_state=42), 
    param_grid=param_grid, 
    cv=5, 
    n_jobs = -1
)  
%time grid_search_forest.fit(X_train, y_train)
y_train_pred = grid_search_forest.predict(X_train)
print('f1_score на обучающем наборе: {:.2f}'.format(metrics.f1_score(y_train, y_train_pred)))
print("accuracy на тестовом наборе: {:.2f}".format(grid_search_forest.score(X_test, y_test)))
y_test_pred = grid_search_forest.predict(X_test)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))
print("Наилучшие значения гиперпараметров: {}".format(grid_search_forest.best_params_))

CPU times: total: 5.92 s
Wall time: 6min 39s
f1_score на обучающем наборе: 0.94
accuracy на тестовом наборе: 0.80
f1_score на тестовом наборе: 0.82
Наилучшие значения гиперпараметров: {'max_depth': 20, 'min_samples_leaf': 5, 'n_estimators': 140}


### <center> **RandomizedSearchCV**

In [82]:
param_distributions = {'penalty': ['l2', 'none'] ,
              'solver': ['lbfgs', 'sag'],
               'C': list(np.linspace(0.01, 1, 10, dtype=float))},
            
random_search = RandomizedSearchCV(
    estimator=linear_model.LogisticRegression(random_state=42, max_iter=50), 
    param_distributions=param_distributions, 
    cv=5, 
    n_iter = 10, 
    n_jobs = -1
)  
%time random_search.fit(X_train, y_train) 
print("accuracy на тестовом наборе: {:.2f}".format(random_search.score(X_test, y_test)))
y_test_pred = random_search.predict(X_test)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))
print("Наилучшие значения гиперпараметров: {}".format(random_search.best_params_))

CPU times: total: 4.81 s
Wall time: 1min 32s
accuracy на тестовом наборе: 0.76
f1_score на тестовом наборе: 0.78
Наилучшие значения гиперпараметров: {'solver': 'sag', 'penalty': 'l2', 'C': 0.34}


c:\Users\Acer\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


In [83]:
param_distributions = {'n_estimators': list(range(100, 200, 30)),
              'min_samples_leaf': [5, 7],
              'max_depth': [10, 15, 20, 25]
              }
            
random_search_forest = RandomizedSearchCV(
    estimator=ensemble.RandomForestClassifier(random_state=42), 
    param_distributions=param_distributions, 
    cv=5
)  
%time random_search_forest.fit(X_train, y_train) 
y_train_pred = random_search_forest.predict(X_train)
print('f1_score на обучающем наборе: {:.2f}'.format(metrics.f1_score(y_train, y_train_pred)))
print("accuracy на тестовом наборе: {:.2f}".format(random_search_forest.score(X_test, y_test)))
y_test_pred = random_search_forest.predict(X_test)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))
print("Наилучшие значения гиперпараметров: {}".format(random_search_forest.best_params_))

CPU times: total: 3min 12s
Wall time: 3min 51s
f1_score на обучающем наборе: 0.94
accuracy на тестовом наборе: 0.80
f1_score на тестовом наборе: 0.82
Наилучшие значения гиперпараметров: {'n_estimators': 100, 'min_samples_leaf': 5, 'max_depth': 15}


### <center> Hyperopt

linear_model.LogisticRegression

In [84]:
intercepts = [True, False]
solvers1 = ["newton-cg", "lbfgs", ]
solvers2 = ["liblinear", ]
solvers3 = ["saga", ]
penalties1 = ["l2", "none"]
penalties2 = ["l1", "l2"]
penalties3 = ["l1", "elasticnet", "l2"]

case1 = {
        "penalty1": hp.choice("penalty1", penalties1),
        "solver1": hp.choice("solver1", solvers1)
}
case2 = {
        "penalty2": hp.choice("penalty2", penalties2),
        "solver2": hp.choice("solver2", solvers2)
}
case3 = {
        "penalty3": hp.choice("penalty3", penalties3),
        "solver3": hp.choice("solver3", solvers3)
}

search_space = {
    "C": hp.uniform("C", 1, 5),
    "fit_intercept": hp.choice("fit_intercept", intercepts),
    "cases" : hp.choice("cases", [("case1", case1), ("case2", case2), ("case3", case3)])
}

In [85]:
def objective(args):
    C = args["C"]
    fit_intercept = args["fit_intercept"]
    kwds = args["cases"]
    penalty = kwds[1]["penalty1"] if kwds[0] == "case1" else kwds[1]["penalty2"] if kwds[0] == "case2" else kwds[1]["penalty3"]
    solver = kwds[1]["solver1"] if kwds[0] == "case1" else kwds[1]["solver2"] if kwds[0] == "case2" else kwds[1]["solver3"]

    log_reg = linear_model.LogisticRegression(C=C,
                                 fit_intercept=fit_intercept,
                                 penalty=penalty,
                                 solver=solver,
                                 l1_ratio=0.5,
                                 random_state=42)

    log_reg.fit(X_train, y_train)

    print("Hyperparameters : {}".format(args)) 
    print("F1-Score : {}\n".format(log_reg.score(X_test, y_test)))
    

    return -1 * log_reg.score(X_test, y_test) 

In [86]:
trials_obj = hyperopt.Trials()

best_results = hyperopt.fmin(objective,
                             space=search_space,
                             algo=hyperopt.tpe.suggest,
                             trials=trials_obj,
                             max_evals=10)

Hyperparameters : {'C': 2.1037389902076806, 'cases': ('case3', {'penalty3': 'elasticnet', 'solver3': 'saga'}), 'fit_intercept': False}
F1-Score : 0.7603195739014648                         

 10%|█         | 1/10 [00:26<03:59, 26.56s/trial, best loss: -0.7603195739014648]

c:\Users\Acer\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(



Hyperparameters : {'C': 1.3230538079005543, 'cases': ('case3', {'penalty3': 'elasticnet', 'solver3': 'saga'}), 'fit_intercept': True}
F1-Score : 0.758988015978695                                                     

 20%|██        | 2/10 [00:53<03:32, 26.55s/trial, best loss: -0.7603195739014648]

c:\Users\Acer\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(

c:\Users\Acer\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_logistic.py:1165: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(



Hyperparameters : {'C': 4.510573247915898, 'cases': ('case1', {'penalty1': 'l2', 'solver1': 'lbfgs'}), 'fit_intercept': True}
F1-Score : 0.7376830892143809                                                    

 30%|███       | 3/10 [00:55<01:47, 15.36s/trial, best loss: -0.7603195739014648]

...

Hyperparameters : {'C': 3.8745665321383402, 'cases': ('case3', {'penalty3': 'l2', 'solver3': 'saga'}), 'fit_intercept': True}
F1-Score : 0.762982689747004                                                    

 80%|████████  | 8/10 [01:54<00:21, 10.96s/trial, best loss: -0.762982689747004]

c:\Users\Acer\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(

c:\Users\Acer\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_logistic.py:1165: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(



Hyperparameters : {'C': 4.282464723597462, 'cases': ('case2', {'penalty2': 'l2', 'solver2': 'liblinear'}), 'fit_intercept': True}
F1-Score : 0.7416777629826897                                                   

 90%|█████████ | 9/10 [01:56<00:08,  8.27s/trial, best loss: -0.762982689747004]

c:\Users\Acer\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_logistic.py:1165: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l1)
  warnings.warn(



Hyperparameters : {'C': 2.004442906321849, 'cases': ('case2', {'penalty2': 'l1', 'solver2': 'liblinear'}), 'fit_intercept': True}
F1-Score : 0.7549933422103862                                                   

100%|██████████| 10/10 [02:00<00:00, 12.07s/trial, best loss: -0.762982689747004]


In [87]:
print("Best Hyperparameters Settings : {}".format(best_results))
print("\nF1-Score : {}".format(-1 * trials_obj.average_best_error()))

Best Hyperparameters Settings : {'C': 3.10513567574049, 'cases': 2, 'fit_intercept': 0, 'penalty3': 2, 'solver3': 0}

F1-Score : 0.762982689747004


In [88]:
C = best_results["C"]
fit_intercept = intercepts[best_results["fit_intercept"]]
if best_results["cases"] == 0:
    penalty = penalties1[best_results["penalty1"]]
    solver = solvers1[best_results["solver1"]]
elif best_results["cases"] == 1:
    penalty = penalties2[best_results["penalty2"]]
    solver = solvers2[best_results["solver2"]]
elif best_results["cases"] == 2:
    penalty = penalties3[best_results["penalty3"]]
    solver = solvers3[best_results["solver3"]]

print("Наилучшие значения гиперпараметров : {}".format({"C":C,
                                                   "penalty": penalty,
                                                   "fit_intercept": fit_intercept,
                                                   "solver":solver,
                                                  }))

Наилучшие значения гиперпараметров : {'C': 3.10513567574049, 'penalty': 'l2', 'fit_intercept': True, 'solver': 'saga'}


In [89]:
log_reg = linear_model.LogisticRegression(C=C,
                             penalty=penalty,
                             fit_intercept=fit_intercept,
                             solver=solver,
                             random_state=42)

log_reg.fit(X_train, y_train)

print("Test  F1-Score : {}".format(log_reg.score(X_test, y_test)))
print("Train F1-Score : {}".format(log_reg.score(X_train, y_train)))

Test  F1-Score : 0.762982689747004
Train F1-Score : 0.86


c:\Users\Acer\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


ensemble.RandomForestClassifier

In [90]:
# зададим пространство поиска гиперпараметров
space={'n_estimators': hp.quniform('n_estimators', 100, 300, 10),
       'max_depth' : hp.quniform('max_depth', 15, 40, 1),
       'min_samples_leaf': hp.quniform('min_samples_leaf', 3, 7, 1)
      }

In [91]:
# зафксируем random_state
random_state = 42
def hyperopt_rf(params, cv=5, X=X_train, y=y_train, random_state=random_state):
    # функция получает комбинацию гиперпараметров в "params"
    params = {'n_estimators': int(params['n_estimators']), 
              'max_depth': int(params['max_depth']), 
             'min_samples_leaf': int(params['min_samples_leaf'])
              }
  
    # используем эту комбинацию для построения модели
    RandomForest_model = ensemble.RandomForestClassifier(**params, random_state=random_state)

    # обучаем модель
    RandomForest_model.fit(X, y)
    score = metrics.f1_score(y, RandomForest_model.predict(X))
    
    # обучать модель можно также с помощью кросс-валидации
    # применим  cross validation с тем же количеством фолдов
    # score = cross_val_score(model, X, y, cv=cv, scoring="f1", n_jobs=-1).mean()

    # метрику необходимо минимизировать, поэтому ставим знак минус
    return -score

In [92]:
%%time
# начинаем подбор гиперпараметров
trials = Trials() # используется для логирования результатов

best=fmin(hyperopt_rf, # наша функция 
          space=space, # пространство гиперпараметров
          algo=tpe.suggest, # алгоритм оптимизации, установлен по умолчанию, задавать необязательно
          max_evals=20, # максимальное количество итераций
          trials=trials, # логирование результатов
          rstate=np.random.default_rng(random_state)# фиксируем для повторяемости результата
         )
print("Наилучшие значения гиперпараметров {}".format(best))

100%|██████████| 20/20 [02:18<00:00,  6.91s/trial, best loss: -0.9779546846295162]
Наилучшие значения гиперпараметров {'max_depth': 22.0, 'min_samples_leaf': 3.0, 'n_estimators': 200.0}
CPU times: total: 2min 7s
Wall time: 2min 18s


In [93]:
# рассчитаем точность для тестовой выборки
RandomForest_model = ensemble.RandomForestClassifier(
    random_state=random_state, 
    n_estimators=int(best['n_estimators']),
    max_depth=int(best['max_depth']),
    min_samples_leaf=int(best['min_samples_leaf'])
)
RandomForest_model.fit(X_train, y_train)
y_train_pred = RandomForest_model.predict(X_train)
print('f1_score на обучающем наборе: {:.2f}'.format(metrics.f1_score(y_train, y_train_pred)))
print("accuracy на тестовом наборе: {:.2f}".format(RandomForest_model.score(X_test, y_test)))
y_test_pred = RandomForest_model.predict(X_test)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))

f1_score на обучающем наборе: 0.98
accuracy на тестовом наборе: 0.81
f1_score на тестовом наборе: 0.83


## <center> Optuna

ensemble.RandomForestClassifier

In [94]:
def optuna_rf(trial):
  # задаем пространства поиска гиперпараметров
  n_estimators = trial.suggest_int('n_estimators', 100, 200, 1)
  max_depth = trial.suggest_int('max_depth', 10, 30, 1)
  min_samples_leaf = trial.suggest_int('min_samples_leaf', 2, 10, 1)

  # создаем модель
  Optuna_model_RF = ensemble.RandomForestClassifier(n_estimators=n_estimators,
                                          max_depth=max_depth,
                                          min_samples_leaf=min_samples_leaf,
                                          random_state=random_state)
  # обучаем модель
  Optuna_model_RF.fit(X_train, y_train)
  score = metrics.f1_score(y_train, Optuna_model_RF.predict(X_train))

  return score

In [95]:
%%time
# cоздаем объект исследования
# можем напрямую указать, что нам необходимо максимизировать метрику direction="maximize"
study = optuna.create_study(study_name="RandomForestClassifier", direction="maximize")
# ищем лучшую комбинацию гиперпараметров n_trials раз
study.optimize(optuna_rf, n_trials=20)

[I 2023-05-12 04:16:26,711] A new study created in memory with name: RandomForestClassifier
[I 2023-05-12 04:16:34,850] Trial 0 finished with value: 0.9573170731707317 and parameters: {'n_estimators': 195, 'max_depth': 18, 'min_samples_leaf': 4}. Best is trial 0 with value: 0.9573170731707317.
[I 2023-05-12 04:16:40,754] Trial 1 finished with value: 0.906087488528602 and parameters: {'n_estimators': 193, 'max_depth': 20, 'min_samples_leaf': 9}. Best is trial 0 with value: 0.9573170731707317.
[I 2023-05-12 04:16:46,397] Trial 2 finished with value: 0.9326393141457441 and parameters: {'n_estimators': 169, 'max_depth': 15, 'min_samples_leaf': 6}. Best is trial 0 with value: 0.9573170731707317.
[I 2023-05-12 04:16:51,620] Trial 3 finished with value: 0.901038485033598 and parameters: {'n_estimators': 177, 'max_depth': 16, 'min_samples_leaf': 10}. Best is trial 0 with value: 0.9573170731707317.
[I 2023-05-12 04:16:57,516] Trial 4 finished with value: 0.9895705521472393 and parameters: {'n_e

CPU times: total: 1min 59s
Wall time: 2min 27s


In [96]:
# выводим результаты на обучающей выборке
print("Наилучшие значения гиперпараметров {}".format(study.best_params))
print("f1_score на обучающем наборе: {:.2f}".format(study.best_value))

Наилучшие значения гиперпараметров {'n_estimators': 162, 'max_depth': 23, 'min_samples_leaf': 2}
f1_score на обучающем наборе: 0.99


In [97]:

model3 = ensemble.RandomForestClassifier(**study.best_params,random_state=random_state, )
model3.fit(X_train, y_train)
y_train_pred = model3.predict(X_train)
print("accuracy на тестовом наборе: {:.2f}".format(model3.score(X_test, y_test)))
y_test_pred = model3.predict(X_test)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))
print('f1_score на обучающем наборе: {:.2f}'.format(metrics.f1_score(y_train, y_train_pred)))

accuracy на тестовом наборе: 0.80
f1_score на тестовом наборе: 0.82
f1_score на обучающем наборе: 0.99


linear_model.LogisticRegression

In [101]:
def objective_RF(trial):
    penalty = trial.suggest_categorical("penalty", ["l1", "l2"])
    tol = trial.suggest_float("tol", 0.0001, 0.01, log=True)
    C = trial.suggest_float("C", 1.0, 10.0, log=True)
    intercept = trial.suggest_categorical("fit_intercept", [True, False])
    solver = trial.suggest_categorical("solver", ["liblinear", "saga"])

    ## Create Model
    classifier = linear_model.LogisticRegression(penalty=penalty,
                                    tol=tol,
                                    C=C,
                                    fit_intercept=intercept,
                                    solver=solver,
                                    multi_class="auto",
                                   )
    ## Fit Model
    classifier.fit(X_train, y_train)

    return classifier.score(X_test, y_test) 

In [102]:
%%time

study3 = optuna.create_study(study_name="LogisticRegression", direction="maximize")
study3.optimize(objective_RF, n_trials=15)

[I 2023-05-12 04:32:37,651] A new study created in memory with name: LogisticRegression
[I 2023-05-12 04:32:40,864] Trial 0 finished with value: 0.7443408788282291 and parameters: {'penalty': 'l2', 'tol': 0.0023043057052372475, 'C': 2.4772232180378064, 'fit_intercept': False, 'solver': 'liblinear'}. Best is trial 0 with value: 0.7443408788282291.
c:\Users\Acer\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
[I 2023-05-12 04:33:16,319] Trial 1 finished with value: 0.762982689747004 and parameters: {'penalty': 'l1', 'tol': 0.0024338542508799623, 'C': 7.924200177089832, 'fit_intercept': True, 'solver': 'saga'}. Best is trial 1 with value: 0.762982689747004.
[I 2023-05-12 04:33:24,036] Trial 2 finished with value: 0.7217043941411452 and parameters: {'penalty': 'l1', 'tol': 0.0009405308114384096, 'C': 9.052887291463513, 'fit_intercept': False, 'sol

CPU times: total: 2min 45s
Wall time: 4min 28s


In [104]:
print("Best Params : {}".format(study3.best_params))

print("\nf1_score : {}".format(study3.best_value))

Best Params : {'penalty': 'l1', 'tol': 0.0006371145680922662, 'C': 1.0147456174560026, 'fit_intercept': False, 'solver': 'saga'}

f1_score : 0.7643142476697736


In [107]:
classifier = linear_model.LogisticRegression(**study3.best_params, multi_class="auto")

classifier.fit(X_train, y_train)

y_train_pred = classifier.predict(X_train)
print("accuracy на тестовом наборе: {:.2f}".format(classifier.score(X_test, y_test)))
y_test_pred = classifier.predict(X_test)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))
print('f1_score на обучающем наборе: {:.2f}'.format(metrics.f1_score(y_train, y_train_pred)))

accuracy на тестовом наборе: 0.76
f1_score на тестовом наборе: 0.79
f1_score на обучающем наборе: 0.86


c:\Users\Acer\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
